In [28]:
import os
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass

In [8]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Your API key and Custom Search Engine ID
api_key = ''
cse_id = 'a6ad5804b25234755'

# Companies and keywords list
companies = [
    "BlackRock", "Vanguard", "BNY Mellon", "Fidelity Investments", "Morgan Stanley"
    , "JPMorgan Chase & Co.", "Goldman Sachs Group, Inc.",
    "Citigroup Inc.", "Wells Fargo & Company", "Bank of America Corporation",
    "UBS Group AG", "Deutsche Bank AG", "HSBC Holdings plc",
    "Charles Schwab Corporation", "Barclays PLC"
]

keywords = [
    'reports', 'declares', 'agrees', 'announces', 'join', 'welcome', 'unveils',
    'introduces', 'sees', 'expands', 'brings', 'present', 'transform', 'launches','teams'
    'increases', 'earns','appoints','affliates','leads'
]

# Initialize an empty DataFrame to store the press releases
press_releases_df = pd.DataFrame(columns=['company', 'title', 'snippet', 'link', 'date'])

# Function to search press releases for a company with given keywords
def search_google_press_releases(api_key, cse_id, company, keywords):
    for keyword in keywords:
        query = f"{company} {keyword}"
        url = "https://www.googleapis.com/customsearch/v1"
        params = {
            'key': api_key,
            'cx': cse_id,
            'q': query,
        }
        
        # Making the request
        response = requests.get(url, params=params)
        search_results = response.json()
        
        # Extract information from the results
        for item in search_results.get('items', []):
            press_releases_df.loc[len(press_releases_df.index)] = [
                company,
                item.get('title'),
                item.get('snippet'),
                item.get('link'),
                'Not provided by API'  # Date extraction may require visiting the link
            ]
        
        # Pause between requests to respect rate limits
        time.sleep(1)

# Iterate over each company and perform searches
for company in companies:
    search_google_press_releases(api_key, cse_id, company, keywords)

# Check the first few entries in the DataFrame
print(press_releases_df.head())


     company                                              title  \
0  BlackRock  BlackRock continues lowering support for envir...   
1  BlackRock  EXCLUSIVE BlackRock backs 3 dissidents to shak...   
2  BlackRock  BlackRock files for bitcoin ETF in push into c...   
3  BlackRock  Nasdaq refiles BlackRock's bitcoin ETF applica...   
4  BlackRock  BlackRock eyes banking rout as chance for grow...   

                                             snippet  \
0  Aug 23, 2023 ... Aug 23 (Reuters) - Top asset ...   
1  May 25, 2021 ... BlackRock, Exxon Mobil Corp's...   
2  Jun 15, 2023 ... They were last at $25,506 on ...   
3  Jul 3, 2023 ... Nasdaq refiled an application ...   
4  Apr 14, 2023 ... The world's largest asset man...   

                                                link                 date  
0  https://www.reuters.com/business/finance/black...  Not provided by API  
1  https://www.reuters.com/business/energy/exclus...  Not provided by API  
2  https://www.reuters.com/busin

In [6]:
# Save the DataFrame to a CSV file
press_releases_df.to_csv('press_releases_data.csv', index=False)

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Your API key and Custom Search Engine ID
api_key = ''
cse_id = 'a6ad5804b25234755'

# Companies and keywords list
companies = [
    "BlackRock", "Vanguard", "BNY Mellon", "Fidelity Investments", "Morgan Stanley"
    , "JPMorgan Chase & Co.", "Goldman Sachs Group, Inc.",
    "Citigroup Inc.", "Wells Fargo & Company", "Bank of America Corporation",
    "UBS Group AG", "Deutsche Bank AG", "HSBC Holdings plc",
    "Charles Schwab Corporation", "Barclays PLC"
]

keywords = [
    'reports', 'declares', 'agrees', 'announces', 'join', 'welcome', 'unveils',
    'introduces', 'sees', 'expands', 'brings', 'present', 'transform', 'launches','teams'
    'increases', 'earns','appoints','affliates','leads'
]

# Initialize an empty DataFrame to store the press releases
press_releases_df = pd.DataFrame(columns=['company', 'title', 'extended_snippet', 'link', 'date'])

def extract_date(snippet):
    # Define a regex pattern for dates in the format 'Aug 23, 2023'
    date_pattern = r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4}'
    match = re.search(date_pattern, snippet)
    if match:
        return match.group(0)  # Return the matched date string
    else:
        return 'Not Available'  # Return a placeholder if no date is found

# Function to scrape the full content from a link
def scrape_content(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Example: Attempt to find the main article content
        content_area = soup.find('div', class_='article-content')  # Adjust based on actual structure
        if content_area:
            return content_area.get_text(strip=True)
        else:
            return "Content could not be retrieved."
    except Exception as e:
        print(f"Error scraping {link}: {e}")
        return "Content could not be retrieved."

# Function to search press releases for a company with given keywords
def search_google_press_releases(api_key, cse_id, company, keywords):
    for keyword in keywords:
        query = f"{company} {keyword}"
        url = "https://www.googleapis.com/customsearch/v1"
        params = {'key': api_key, 'cx': cse_id, 'q': query}
        
        # Making the request
        response = requests.get(url, params=params)
        search_results = response.json()
        
        # Extract information from the results
        # Updated part of the script where information from the results is extracted
    # Apply the extract_date function to the 'snippet' column when adding rows to the DataFrame
    for item in search_results.get('items', []):
        date = extract_date(item.get('snippet'))  # Extract date from the snippet
        press_releases_df.loc[len(press_releases_df.index)] = [
            company,
            item.get('title'),
            item.get('snippet')[:500],  # Optionally truncate the snippet
            item.get('link'),
            date  # Use the extracted date
        ]

        
        # Pause between requests to respect rate limits
        time.sleep(1)

# Iterate over each company and perform searches
for company in companies:
    search_google_press_releases(api_key, cse_id, company, keywords)

# Save the DataFrame to a CSV file
press_releases_df.to_csv('press_releases_data_500.csv', index=False)

# Optionally, print the first few entries in the DataFrame to verify
press_releases_df.head()

,company,title,extended_snippet,link,date
0,BlackRock,BlackRock leads $30 million round for biofuels...,"Dec 20, 2010 ... One of LS9's selling points i...",https://www.reuters.com/article/idUS1120082164...,"Dec 20, 2010"
1,BlackRock,"Executives leave BlackRock, ISS to lead govern...","Oct 17, 2023 ... Jessica McDougall, a former d...",https://www.reuters.com/business/finance/execu...,"Oct 17, 2023"
2,BlackRock,BlackRock debuts U.S. ETF for China-listed sto...,"Jun 15, 2016 ... Martin Small, who leads the B...",https://www.reuters.com/article/us-blackrock-c...,"Jun 15, 2016"
3,BlackRock,"BlackRock CEO Fink warns of financial risks, p...","Mar 15, 2023 ... BlackRock Inc Chief Executive...",https://www.reuters.com/business/finance/black...,"Mar 15, 2023"
4,BlackRock,BlackRock taps former Obama aide to lead susta...,"Mar 30, 2021 ... Top asset manager BlackRock I...",https://www.reuters.com/article/blackrock-sust...,"Mar 30, 2021"


In [15]:
press_releases_df.shape

(40, 5)

In [16]:
# Your API key and Custom Search Engine ID
api_key = ''
cse_id = 'a6ad5804b25234755'

# Companies and keywords list
companies = [
    "BlackRock", "Vanguard", "BNY Mellon", "Fidelity Investments", "Morgan Stanley"
    , "JPMorgan Chase & Co.", "Goldman Sachs Group, Inc.",
    "Citigroup Inc.", "Wells Fargo & Company", "Bank of America Corporation",
    "UBS Group AG", "Deutsche Bank AG", "HSBC Holdings plc",
    "Charles Schwab Corporation", "Barclays PLC"
]

keywords = [
    'reports', 'declares', 'agrees', 'announces', 'join', 'welcome', 'unveils',
    'introduces', 'sees', 'expands', 'brings', 'present', 'transform', 'launches','teams'
    'increases', 'earns','appoints','affliates','leads'
]

# Initialize an empty DataFrame to store the press releases
press_releases_df = pd.DataFrame(columns=['company', 'title', 'snippet', 'link', 'date'])

def extract_date(snippet):
    # Define a regex pattern for dates in the format 'Aug 23, 2023'
    date_pattern = r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4}'
    match = re.search(date_pattern, snippet)
    if match:
        return match.group(0)  # Return the matched date string
    else:
        return 'Not Available'  # Return a placeholder if no date is found

# Function to search press releases for a company with given keywords
def search_google_press_releases(api_key, cse_id, company, keywords):
    for keyword in keywords:
        query = f"{company} {keyword}"
        url = "https://www.googleapis.com/customsearch/v1"
        params = {
            'key': api_key,
            'cx': cse_id,
            'q': query,
        }
        
        # Making the request
        response = requests.get(url, params=params)
        search_results = response.json()
        
        # Extract information from the results
        for item in search_results.get('items', []):
            date = extract_date(item.get('snippet'))  # Extract date from the snippet
            press_releases_df.loc[len(press_releases_df.index)] = [
                company,
                item.get('title'),
                item.get('snippet'),
                item.get('link'),
                date  # Date extraction may require visiting the link
            ]
        
        # Pause between requests to respect rate limits
        time.sleep(1)

# Iterate over each company and perform searches
for company in companies:
    search_google_press_releases(api_key, cse_id, company, keywords)

# Save the DataFrame to a CSV file
press_releases_df.to_csv('press_releases_data_new.csv', index=False)

# Check the first few entries in the DataFrame
press_releases_df.head()


,company,title,snippet,link,date
0,BlackRock,BlackRock continues lowering support for envir...,"Aug 23, 2023 ... Aug 23 (Reuters) - Top asset ...",https://www.reuters.com/business/finance/black...,"Aug 23, 2023"
1,BlackRock,EXCLUSIVE BlackRock backs 3 dissidents to shak...,"May 25, 2021 ... BlackRock, Exxon Mobil Corp's...",https://www.reuters.com/business/energy/exclus...,"May 25, 2021"
2,BlackRock,BlackRock files for bitcoin ETF in push into c...,"Jun 15, 2023 ... They were last at $25,506 on ...",https://www.reuters.com/business/finance/black...,"Jun 15, 2023"
3,BlackRock,Nasdaq refiles BlackRock's bitcoin ETF applica...,"Jul 3, 2023 ... Nasdaq refiled an application ...",https://www.reuters.com/technology/nasdaq-refi...,"Jul 3, 2023"
4,BlackRock,BlackRock eyes banking rout as chance for grow...,"Apr 14, 2023 ... The world's largest asset man...",https://www.reuters.com/business/finance/black...,"Apr 14, 2023"


In [20]:
press_releases_df['date'] = pd.to_datetime(press_releases_df['date'], errors='coerce')

# Extract month and handle missing values
press_releases_df['Month'] = press_releases_df['date'].dt.month_name().fillna('Unknown')

# For year, temporarily fill NaT with a placeholder, convert to integer, then replace placeholder
press_releases_df['Year'] = press_releases_df['date'].dt.year.fillna(-1).astype(int)  # Convert year to int, using -1 for NaT
press_releases_df['Year'] = press_releases_df['Year'].replace(-1, 'Unknown')  # Replace -1 back with 'Unknown'

In [17]:
def generate_press_release_context(row):
    # Customizable template structure
    template = '''
    Press Release Summary for {Company}: On {Date}, {Company} announced a significant development, titled "{Title}". A brief overview of the announcement is as follows: {Snippet}. For more detailed information, the full press release can be accessed through the link: {Link}.
    '''
    
    # Extracting 'Not Available' dates and replacing them with a more natural phrase
    date_text = row['date'] if row['date'] != 'Not Available' else "a date not specified in the snippet"
    
    # Placeholders and replacements
    replacements = {
        'Company': row['company'],
        'Date': date_text,
        'Title': row['title'],
        'Snippet': row['snippet'],
        'Link': row['link']
    }

    # Generate the context string
    context = template.format(**replacements)

    return context.strip()

# Apply the context generation function to each row in the DataFrame
press_releases_df['Context'] = press_releases_df.apply(generate_press_release_context, axis=1)

# Optionally, you can drop columns or modify the DataFrame further as needed
# For example, to focus only on the newly created context and the company name:
press_df = press_releases_df[['company', 'Context']]

# Display the first few entries in the DataFrame to verify the context generation
press_df.head()


,company,Context
0,BlackRock,Press Release Summary for BlackRock: On Aug 23...
1,BlackRock,Press Release Summary for BlackRock: On May 25...
2,BlackRock,Press Release Summary for BlackRock: On Jun 15...
3,BlackRock,"Press Release Summary for BlackRock: On Jul 3,..."
4,BlackRock,Press Release Summary for BlackRock: On Apr 14...


In [19]:
press_releases_df.head()

,company,title,snippet,link,date,Context,Month,Year
0,BlackRock,BlackRock continues lowering support for envir...,"Aug 23, 2023 ... Aug 23 (Reuters) - Top asset ...",https://www.reuters.com/business/finance/black...,2023-08-23,Press Release Summary for BlackRock: On Aug 23...,August,2023.0
1,BlackRock,EXCLUSIVE BlackRock backs 3 dissidents to shak...,"May 25, 2021 ... BlackRock, Exxon Mobil Corp's...",https://www.reuters.com/business/energy/exclus...,2021-05-25,Press Release Summary for BlackRock: On May 25...,May,2021.0
2,BlackRock,BlackRock files for bitcoin ETF in push into c...,"Jun 15, 2023 ... They were last at $25,506 on ...",https://www.reuters.com/business/finance/black...,2023-06-15,Press Release Summary for BlackRock: On Jun 15...,June,2023.0
3,BlackRock,Nasdaq refiles BlackRock's bitcoin ETF applica...,"Jul 3, 2023 ... Nasdaq refiled an application ...",https://www.reuters.com/technology/nasdaq-refi...,2023-07-03,"Press Release Summary for BlackRock: On Jul 3,...",July,2023.0
4,BlackRock,BlackRock eyes banking rout as chance for grow...,"Apr 14, 2023 ... The world's largest asset man...",https://www.reuters.com/business/finance/black...,2023-04-14,Press Release Summary for BlackRock: On Apr 14...,April,2023.0


In [22]:
press_releases_df.head()

,company,title,snippet,link,date,Context,Month,Year
0,BlackRock,BlackRock continues lowering support for envir...,"Aug 23, 2023 ... Aug 23 (Reuters) - Top asset ...",https://www.reuters.com/business/finance/black...,2023-08-23,Press Release Summary for BlackRock: On Aug 23...,August,2023
1,BlackRock,EXCLUSIVE BlackRock backs 3 dissidents to shak...,"May 25, 2021 ... BlackRock, Exxon Mobil Corp's...",https://www.reuters.com/business/energy/exclus...,2021-05-25,Press Release Summary for BlackRock: On May 25...,May,2021
2,BlackRock,BlackRock files for bitcoin ETF in push into c...,"Jun 15, 2023 ... They were last at $25,506 on ...",https://www.reuters.com/business/finance/black...,2023-06-15,Press Release Summary for BlackRock: On Jun 15...,June,2023
3,BlackRock,Nasdaq refiles BlackRock's bitcoin ETF applica...,"Jul 3, 2023 ... Nasdaq refiled an application ...",https://www.reuters.com/technology/nasdaq-refi...,2023-07-03,"Press Release Summary for BlackRock: On Jul 3,...",July,2023
4,BlackRock,BlackRock eyes banking rout as chance for grow...,"Apr 14, 2023 ... The world's largest asset man...",https://www.reuters.com/business/finance/black...,2023-04-14,Press Release Summary for BlackRock: On Apr 14...,April,2023


In [17]:
df = pd.read_csv('press_releases_data_new.csv')

In [18]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Extract month and handle missing values
df['Month'] = df['date'].dt.month_name().fillna('Unknown')

# For year, temporarily fill NaT with a placeholder, convert to integer, then replace placeholder
df['Year'] = df['date'].dt.year.fillna(-1).astype(int)  # Convert year to int, using -1 for NaT
df['Year'] = df['Year'].replace(-1, 'Unknown')  # Replace -1 back with 'Unknown'

In [19]:
df1 = df[df['Year'] != 'Unknown']

In [20]:
df1["Month-Year"] = df1["Month"] + ' ' + df1["Year"].astype(str)

C:\Users\Admin\AppData\Local\Temp\ipykernel_1428\1186889055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Month-Year"] = df1["Month"] + ' ' + df1["Year"].astype(str)


In [23]:
df1.drop(['link','date','Month','Year'],axis=1)

,company,title,snippet,Month-Year
0,BlackRock,BlackRock continues lowering support for envir...,"Aug 23, 2023 ... Aug 23 (Reuters) - Top asset ...",August 2023
1,BlackRock,EXCLUSIVE BlackRock backs 3 dissidents to shak...,"May 25, 2021 ... BlackRock, Exxon Mobil Corp's...",May 2021
2,BlackRock,BlackRock files for bitcoin ETF in push into c...,"Jun 15, 2023 ... They were last at $25,506 on ...",June 2023
3,BlackRock,Nasdaq refiles BlackRock's bitcoin ETF applica...,"Jul 3, 2023 ... Nasdaq refiled an application ...",July 2023
4,BlackRock,BlackRock eyes banking rout as chance for grow...,"Apr 14, 2023 ... The world's largest asset man...",April 2023
...,...,...,...,...
953,JPMorgan Chase & Co.,Exclusive: JPMorgan CEO says too early to decl...,"Feb 8, 2023 ... The chief executive of JPMorga...",February 2023
954,JPMorgan Chase & Co.,Exclusive: JPMorgan employees gripe about Dimo...,"Apr 27, 2023 ... NEW YORK, April 27 (Reuters) ...",April 2023
956,JPMorgan Chase & Co.,JPMorgan cutting about 500 jobs this week | Re...,"May 26, 2023 ... May 26 (Reuters) - JPMorgan C...",May 2023
957,JPMorgan Chase & Co.,JPMorgan Chase to spend $200 million on carbon...,"May 23, 2023 ... U.S. bank JPMorgan Chase & Co...",May 2023


In [35]:

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("")

llm = ChatGoogleGenerativeAI(model="gemini-pro")

def create_llm_snippet(title, snippet):
    prompt = f"""Title: {title}
Snippet: {snippet}
Create a coherent sentence and make it look like a proper news summary in 50 words. 
You can use your training data or go through the internet resources to create a coherent sentence out of this."""
    
    try:
        response = llm.invoke(prompt)
        # Assuming the response has a text field with the result
        return response.content
    except Exception as e:
        print(f"Error during model invocation: {e}")
        # Return some placeholder text or re-raise the exception
        return "Error generating snippet"



In [36]:
df1['LLM snippet'] = df1.apply(lambda row: create_llm_snippet(row['title'], row['snippet']), axis=1)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Error during model invocation: index: 0
finish_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: HIGH
}
citation_metadata {
  citation_sources {
    start_index: 836
    end_index: 983
    uri: "https://github.com/GoodJob567/ffxiv-best-gearing"
    license_: ""
  }
  citation_sources {
    start_index: 986
    end_index: 1106
    uri: "https://github.com/paule32/dBase4LinuxIDE"
    license_: ""
  }
  citation_sources {
    start_index: 992
    end_index: 1133
    uri: "https://github.com/RaidenXin/spring-boot-test"
    license_: ""
  }
  citation_sources {
    start_index: 1209
    end_index: 1446
    uri: "https://issuu.com/cisr-journal/docs/23.2"
    license_: ""
  }
  citation_sources {
    st

C:\Users\Admin\AppData\Local\Temp\ipykernel_1428\4052592673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['LLM snippet'] = df1.apply(lambda row: create_llm_snippet(row['title'], row['snippet']), axis=1)


In [41]:
df1.to_csv('LLM_snippet.csv', index = False)

In [43]:
df2 = df1.drop(['link','date','Month','Year'],axis=1)

In [46]:
df2.drop(['title','snippet'], inplace=True, axis= 1)

In [49]:
df_final = df2[df2['LLM snippet'] != "Error generating snippet"]

In [51]:
df_final.to_csv('final_news_press.csv',index=False)

In [55]:
df_final.head()

,company,Month-Year,LLM snippet
0,BlackRock,August 2023,"BlackRock, the world's largest asset manager, ..."
1,BlackRock,May 2021,"BlackRock, Exxon Mobil's second largest shareh..."
2,BlackRock,June 2023,Investment management firm BlackRock has filed...
3,BlackRock,July 2023,Nasdaq has reapplied for approval from the SEC...
4,BlackRock,April 2023,"Despite a decline in profits, BlackRock, the w..."


In [57]:
def generate_press_release_context(row):
    # Customizable template structure
    template = '''
    Latest information for {Company} in {MonthYear}. {Company} disclosed noteworthy information: {LLMSnippet}.
    '''
    
    # Placeholders and replacements
    replacements = {
        'Company': row['company'],
        'MonthYear': row['Month-Year'],
        'LLMSnippet': row['LLM snippet']
    }

    # Generate the context string
    context = template.format(**replacements)

    return context.strip()

# Apply the context generation function to each row in the DataFrame
df_final['Context'] = df_final.apply(generate_press_release_context, axis=1)

# Display the first few entries in the DataFrame to verify the context generation
df_final.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_1428\4066387647.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Context'] = df_final.apply(generate_press_release_context, axis=1)


,company,Month-Year,LLM snippet,Context
0,BlackRock,August 2023,"BlackRock, the world's largest asset manager, ...",Latest information for BlackRock in August 202...
1,BlackRock,May 2021,"BlackRock, Exxon Mobil's second largest shareh...",Latest information for BlackRock in May 2021. ...
2,BlackRock,June 2023,Investment management firm BlackRock has filed...,Latest information for BlackRock in June 2023....
3,BlackRock,July 2023,Nasdaq has reapplied for approval from the SEC...,Latest information for BlackRock in July 2023....
4,BlackRock,April 2023,"Despite a decline in profits, BlackRock, the w...",Latest information for BlackRock in April 2023...


In [63]:
df_big = df_final[['Context']]
df_big.to_csv('final_context.csv',index=False)